# Motif Example Classifier  
A classifier CNN model, trained to classify novel examples of a given motif. 

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import os

### GPU Setup 

In [ ]:
# Check that TensorFlow can see the GPU
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

In [ ]:
# Set TensorFlow option to grow GPU memory allocation when required
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Data Augmentation

Store raw data at `./data/positive` and `./data/negative` and augment in place

### Make Spectrograms

Save spectrograms into `./data/train/{class}/{name}` and `./data/test/{class}/{name}`

### Load Spectrograms as Data Set

Data lives in `./data/train`

In [ ]:
# Sort spectrograms into directories
# Use if loading directory of spectrograms

import os
import shutil
val_split = 8

for index, item in enumerate(os.listdir("./spec/negative/")):
    if index % val_split == 0: shutil.move("./spec/negative/" + item, "./test_data/negative/" + item)
    else: shutil.move("./spec/negative/" + item, "./train_data/negative/" + item)

for index, item in enumerate(os.listdir("./spec/positive/")):
    if index % val_split == 0: shutil.move("./spec/positive/" + item, "./test_data/positive/" + item)
    else: shutil.move("./spec/positive/" + item, "./train_data/positive/" + item)
        

In [ ]:
train_directory = './train_data/'
test_directory = './test_data/'
image_size = (360,225) ## Specify the size of the spectrogram images

# Build the train dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_directory, labels='inferred', label_mode='int', image_size=image_size, seed=123)

# Build the test dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_directory, labels='inferred', label_mode='int', image_size=image_size, seed=123)


class_names = train_ds.class_names
print(class_names)

In [ ]:
# Straight from the microsoft tutorial 
# shows the spectrograms with their labels
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(2):
    for i in range(9):
        ax = plt.subplot(3, 5, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

### Build the Model
Define the layers using Keras

In [ ]:
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters, 3, strides=strides,
                            padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3, strides=1,
                            padding="same", use_bias=False),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters, 1, strides=strides,
                                  padding="same", use_bias=False),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, 7, strides=2, input_shape=[360, 225, 3],
                              padding="same", use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"))
prev_filter = 64
for filters in [64]*3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filter else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(2, activation="softmax"))

In [ ]:
learning_rate = 0.02

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.SGD(learning_rate)
metrics = ['acc']
model.compile(optimizer, loss_fn, metrics)
model.summary()

In [ ]:
# Set the epochs
epochs = 5
print('\nFitting:')

# Train the model.
history = model.fit(train_ds, validation_data=test_ds, epochs=epochs, batch_size=16)

### Train the Model

In [ ]:
model.save("./model.tf")

### Evaluate the Model
Run the model throught the test data at `./data/test` and calculate accuracy.